In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.saving import register_keras_serializable
from sklearn.preprocessing import StandardScaler
import joblib
import os

# === 1. Load feature data ===
df_yes = pd.read_csv("dataset/flood_yes.csv")
df_no = pd.read_csv("dataset/flood_no.csv")

data = pd.concat([df_yes, df_no], ignore_index=True)
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

X = data.drop(columns=["flood_binary"]).astype("float32")
y = data["flood_binary"].astype(np.float32)

print(f"✅ Loaded {len(X)} samples | Flood: {y.sum()} | No Flood: {(y == 0).sum()}")

# === 2. Generate pseudo-trust scores in [0.5, 1.5] ===
def generate_flood_trust_score(row):
    rain = row["rainfall"]
    dist = row["distance_from_river"]
    slope = row["slope"]
    elev = row["elevation"]

    rain_factor = np.clip((rain - 60) / 60, 0, 1)  # >60 mm rising flood risk
    dist_factor = np.clip((200 - dist) / 200, 0, 1)  # closer to river = riskier
    slope_factor = np.clip((slope - 2) / 5, 0, 1)  # Steep slope multiplier
    elev_factor = np.clip((9 - elev) / 6, 0, 1)  # Lower elevation = flood-prone

    composite = (
        0.3 * rain_factor +
        0.3 * dist_factor +
        0.2 * slope_factor +
        0.2 * elev_factor
    )
    return 0.5 + composite

mod_labels = X.apply(generate_flood_trust_score, axis=1).values.astype(np.float32)

# === 3. Normalize features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === 4. Define FloodTrustNet ===
@register_keras_serializable()
def floodtrust_activation(x):
    return 0.5 + tf.sigmoid(x)

input_layer = layers.Input(shape=(X.shape[1],))
z = layers.Dense(16, activation='relu')(input_layer)
z = layers.Dense(8, activation='relu')(z)
output = layers.Dense(1, activation=floodtrust_activation)(z)

trustnet = models.Model(inputs=input_layer, outputs=output)
trustnet.compile(optimizer="adam", loss="mse")
trustnet.summary()

# === 5. Train ===
trustnet.fit(X_scaled, mod_labels, batch_size=32, epochs=10, validation_split=0.2)

# === 6. Save ===
os.makedirs("artifacts", exist_ok=True)
trustnet.save("artifacts/floodtrustnet.h5")
joblib.dump(scaler, "artifacts/floodtrust_scaler.pkl")
print("✅ FloodTrustNet model + scaler saved.")


✅ Loaded 144000 samples | Flood: 72000.0 | No Flood: 72000


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 241 (964.00 B)

 Trainable params: 241 (964.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0074 - val_loss: 1.2735e-04
Epoch 2/10
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 1.0941e-04 - val_loss: 6.2125e-05
Epoch 3/10
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 5.4184e-05 - val_loss: 3.9883e-05
Epoch 4/10
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 3.3445e-05 - val_loss: 2.7464e-05
Epoch 5/10
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 2.7128e-05 - val_loss: 2.6635e-05
Epoch 6/10
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 2.3864e-05 - val_loss: 3.8533e-05
Epoch 7/10
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 2.1363e-05 - val_loss: 1.9643e-05
Epoch 8/10
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.9646e-05 - val_loss: 2.2882e-05
Epoch 9/10
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.9219e-05 - val_loss: 1.6997e-05
Epoch 10/10
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.8043e-05 - val_loss: 1.8980e-05


✅ FloodTrustNet model + scaler saved.
